In [1]:
import pandas as pd
import numpy as np
import jieba; jieba.initialize()
from collections import Counter
import re

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/l6/b9fxvmjn0qj4340p4p7mzptw0000gn/T/jieba.cache
Loading model cost 0.701 seconds.
Prefix dict has been built succesfully.


In [2]:
df = pd.read_excel("../data/患者信息汇总20190726.xls",encoding='gbk')
print(df.shape)
df.head()

(64, 3)


,主诉,手术（日期+术式）,生存期
0,头痛恶心伴视物模糊2个月。\n,2017-08-08 神经导航引导下，右额开颅胶质瘤切除术,11个半月
1,右下肢疼痛1月余，伴左下肢活动不灵半个月。,2016.04.11\n原切口入路右侧侧脑室血肿清除术\n2016.04.11\n神经导航下...,11个月
2,言语不清10天余。,2016.10.17术中超声引导下 左侧额叶胶质母细胞瘤切除术\n\n,12个月
3,发现右侧额叶占位病变1个月。,2015.12.04\n冠状切口 右侧额叶胶质瘤切除术\n,12个月
4,精神萎靡伴双下肢无力2天。,2018年4月28日 半冠状切口右额开颅右额占位切除术+颅骨修补术,14个月


# 主诉+词频

In [3]:
# remove trailing chars: '。\n'
df['主诉'] = df['主诉'].map(lambda x: x.rstrip('。\n'))
df.head()

,主诉,手术（日期+术式）,生存期
0,头痛恶心伴视物模糊2个月,2017-08-08 神经导航引导下，右额开颅胶质瘤切除术,11个半月
1,右下肢疼痛1月余，伴左下肢活动不灵半个月,2016.04.11\n原切口入路右侧侧脑室血肿清除术\n2016.04.11\n神经导航下...,11个月
2,言语不清10天余,2016.10.17术中超声引导下 左侧额叶胶质母细胞瘤切除术\n\n,12个月
3,发现右侧额叶占位病变1个月,2015.12.04\n冠状切口 右侧额叶胶质瘤切除术\n,12个月
4,精神萎靡伴双下肢无力2天,2018年4月28日 半冠状切口右额开颅右额占位切除术+颅骨修补术,14个月


In [4]:
# words that should not be split during tokenization
whole_words = {'一过性','发作性','伴','不自主活动','面部','视物'}
for word in whole_words:
    jieba.suggest_freq(word, True)
jieba.del_word('胶质瘤')

In [5]:
words = []

# tokenize each complaint and put all tokens in the same list
for complaint in df['主诉']:
    seg_list = jieba.lcut(complaint, cut_all=False)
    print(repr(complaint))
    print(seg_list)
    print()
    words.extend(seg_list)

'头痛恶心伴视物模糊2个月'
['头痛', '恶心', '伴', '视物', '模糊', '2', '个', '月']

'右下肢疼痛1月余，伴左下肢活动不灵半个月'
['右', '下肢', '疼痛', '1', '月余', '，', '伴', '左', '下肢', '活动', '不灵', '半个', '月']

'言语不清10天余'
['言语', '不清', '10', '天余']

'发现右侧额叶占位病变1个月'
['发现', '右侧', '额叶', '占位', '病变', '1', '个', '月']

'精神萎靡伴双下肢无力2天'
['精神', '萎靡', '伴', '双下肢', '无力', '2', '天']

'头晕、恶心、呕吐1天'
['头晕', '、', '恶心', '、', '呕吐', '1', '天']

'头痛并四肢无力1周'
['头痛', '并', '四肢无力', '1', '周']

'左下肢无力1个月，加重1星期'
['左', '下肢', '无力', '1', '个', '月', '，', '加重', '1', '星期']

'言语障碍3周，头痛半月，头痛加重1周'
['言语', '障碍', '3', '周', '，', '头痛', '半月', '，', '头痛', '加重', '1', '周']

'突发左侧肢体活动不灵，伴头痛头晕恶心呕吐3小时'
['突发', '左侧', '肢体', '活动', '不灵', '，', '伴', '头痛', '头晕', '恶心', '呕吐', '3', '小时']

'头痛、呕吐10天'
['头痛', '、', '呕吐', '10', '天']

'突发意识不清7天余'
['突发', '意识', '不清', '7', '天余']

'发作性意识丧失伴肢体抽搐5天，加重1天'
['发作性', '意识', '丧失', '伴', '肢体', '抽搐', '5', '天', '，', '加重', '1', '天']

'头痛头晕15天,摔伤致一过性意识丧失10天余'
['头痛', '头晕', '15', '天', ',', '摔伤', '致', '一过性', '意识', '丧失', '10', '天余']

'头痛、呕吐1个月，发现颅内占位半个月'
['头痛', '、', '呕吐', '1', '个', '月'

In [6]:
def is_stopword(token: str) -> bool:
    """ return if a word is a stop word """
    
    stopwords = {'，', '。', '、', ',', 
                 '小时','天', '月', '周', '年', 
                 '个', '伴', '余天', '无',
                 '左', '右', '左侧', '右侧',
                 '年余','月余','天余','术后',
                 '发现','反复','加重','突发',
                 '半个','半月','不能','余'}
    if token.isdigit():
        return True
    if token in stopwords:
        return True
    return False

In [7]:
c = Counter(words)
high_freq = []
for pair in c.most_common():
    if not is_stopword(pair[0]):
        high_freq.append(pair)

# tokens that appear more than 3 times
high_freq = [x[0] for x in high_freq if x[1] >= 3]
print(high_freq)

['头痛', '肢体', '瘤', '胶质', '呕吐', '头晕', '恶心', '言语', '占位', '无力', '意识', '活动', '下肢', '不灵', '不清', '抽搐', '颅内', '母细胞', '记忆力', '障碍', '发作性', '丧失', '乏力', '上肢']


## 人工调整

In [8]:
# remove tokens that have been covered by others
remove_list = {'肢体', '母细胞', '胶质', '下肢', '不灵', 
               '不清', '颅内', '障碍', '发作性', '丧失', 
               '上肢', '视物', '进行性', '颞叶','颞', '复发'}
high_freq = [x for x in high_freq if x not in remove_list]

# add the two tokens that are probabaly useful but only appear twice
high_freq.append('模糊')
high_freq.append('失禁')
print(high_freq)

['头痛', '瘤', '呕吐', '头晕', '恶心', '言语', '占位', '无力', '意识', '活动', '抽搐', '记忆力', '乏力', '模糊', '失禁']


In [9]:
# create a dictionary that has 
# the tokens to be recognized in text as values,
# and tokens' proper names as keys 
kw_dict = {}

kw_dict['头痛'] = {'头痛'}
kw_dict['胶质瘤手术'] = {'瘤'}
kw_dict['呕吐'] = {'呕吐'}
kw_dict['头晕'] = {'头晕'}
kw_dict['恶心'] = {'恶心'}
kw_dict['言语障碍'] = {'言语'}
kw_dict['占位性病变'] = {'占位'}
kw_dict['肢体无力'] = {'无力','乏力'}
kw_dict['意识障碍'] = {'意识'}
kw_dict['肢体活动不灵'] = {'活动'}
kw_dict['肢体抽搐'] = '抽搐'
kw_dict['记忆力减退'] = '记忆力'
kw_dict['视物模糊'] = '模糊'
kw_dict['大小便失禁'] = '失禁'

kw_dict

{'头痛': {'头痛'},
 '胶质瘤手术': {'瘤'},
 '呕吐': {'呕吐'},
 '头晕': {'头晕'},
 '恶心': {'恶心'},
 '言语障碍': {'言语'},
 '占位性病变': {'占位'},
 '肢体无力': {'乏力', '无力'},
 '意识障碍': {'意识'},
 '肢体活动不灵': {'活动'},
 '肢体抽搐': '抽搐',
 '记忆力减退': '记忆力',
 '视物模糊': '模糊',
 '大小便失禁': '失禁'}

## 创建特征集

In [21]:
one_hot_mtx = np.zeros((len(df), len(kw_dict)))  # 64*14 matrix